<a href="https://colab.research.google.com/github/marcinwolter/DeepLearning_2020/blob/main/mnist_mlp_minimal_optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
"""
Optuna example that optimizes a neural network classifier configuration for the
MNIST dataset using Keras.
In this example, we optimize the validation accuracy of MNIST classification using
Keras. We optimize the filter and kernel size, kernel stride and layer activation.
"""
! pip install optuna

import warnings

from keras.backend import clear_session
from keras.datasets import mnist
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.optimizers import RMSprop

import optuna

In [13]:

N_TRAIN_EXAMPLES = 3000
N_VALID_EXAMPLES = 1000
BATCHSIZE = 128
CLASSES = 10
EPOCHS = 10


In [14]:
def objective(trial):
    # Clear clutter from previous Keras session graphs.
    clear_session()

    (x_train, y_train), (x_valid, y_valid) = mnist.load_data()

    x_train = x_train.reshape(60000, 784)[:N_TRAIN_EXAMPLES].astype("float32") / 255
    x_valid = x_valid.reshape(10000, 784)[:N_VALID_EXAMPLES].astype("float32") / 255
    y_train = y_train[:N_TRAIN_EXAMPLES]
    y_valid = y_valid[:N_VALID_EXAMPLES]
    

    units = int(trial.suggest_uniform("units1", 32,1024))
    dropout = trial.suggest_uniform('dropout1', .00,.50)
    act = trial.suggest_categorical('activation', ['relu','sigmoid','linear'])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)

    model = Sequential()
    model.add(Dense(units, activation='relu', input_shape=(784,)))
    model.add(Dropout(dropout))
    model.add(Dense(units, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(units, activation='relu'))
    model.add(Dropout(dropout))

    model.add(Dense(CLASSES, activation="softmax"))

    # We compile our model with a sampled learning rate.
    model.compile(
        loss="sparse_categorical_crossentropy", optimizer=RMSprop(lr=lr), metrics=["accuracy"]
    )

    model.fit(
        x_train,
        y_train,
        validation_data=(x_valid, y_valid),
        shuffle=True,
        batch_size=BATCHSIZE,
        epochs=EPOCHS,
        verbose=False,
    )

    # Evaluate the model accuracy on the validation set.
    score = model.evaluate(x_valid, y_valid, verbose=0)
    return score[1]


In [15]:



if __name__ == "__main__":
    warnings.warn(
        "Recent Keras release (2.4.0) simply redirects all APIs "
        "in the standalone keras package to point to tf.keras. "
        "There is now only one Keras: tf.keras. "
        "There may be some breaking changes for some workflows by upgrading to keras 2.4.0. "
        "Test before upgrading. "
        "REF:https://github.com/keras-team/keras/releases/tag/2.4.0"
    )
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning:

Recent Keras release (2.4.0) simply redirects all APIs in the standalone keras package to point to tf.keras. There is now only one Keras: tf.keras. There may be some breaking changes for some workflows by upgrading to keras 2.4.0. Test before upgrading. REF:https://github.com/keras-team/keras/releases/tag/2.4.0

[I 2020-11-30 22:31:43,923] A new study created in memory with name: no-name-5de42fee-e943-4a6c-951f-db7a6c7e0afb
[I 2020-11-30 22:31:45,744] Trial 0 finished with value: 0.2919999957084656 and parameters: {'units1': 143.81395835016806, 'dropout1': 0.0761605150299261, 'activation': 'sigmoid', 'lr': 0.08491987333455679}. Best is trial 0 with value: 0.2919999957084656.
[I 2020-11-30 22:31:47,553] Trial 1 finished with value: 0.8880000114440918 and parameters: {'units1': 309.76818856175714, 'dropout1': 0.005000237243033057, 'activation': 'sigmoid', 'lr': 0.00017496766670807844}. Best is trial 1 with valu

Number of finished trials: 100
Best trial:
  Value: 0.9350000023841858
  Params: 
    units1: 713.0171042396482
    dropout1: 0.05668431823113735
    activation: relu
    lr: 0.002122871105209085


## Visualizing the Optimization History

In [16]:
from optuna.visualization import plot_optimization_history

plot_optimization_history(study)

## Visualizing High-dimensional Parameter Relationships

In [17]:
from optuna.visualization import plot_parallel_coordinate

plot_parallel_coordinate(study)

## Visualizing Parameter Relationships

In [18]:
from optuna.visualization import plot_contour

plot_contour(study)

### Selecting Parameters to Visualize

In [25]:
plot_contour(study, params=['units1', 'dropout1', 'lr'])

## Visualizing Individual Parameters

In [20]:
from optuna.visualization import plot_slice

plot_slice(study)

### Selecting Parameters to Visualize

In [21]:
plot_slice(study, params=['units1', 'dropout1'])

## Visualizing Parameter Importances

In [22]:
from optuna.visualization import plot_param_importances

plot_param_importances(study)